Лабораторна робота 6

In [1]:
import numpy as np
import pandas as pd

In [2]:
def calcRating(english, ukrainian, math):
    return 0.4 * math + 0.3 * english + 0.3 * ukrainian

def checkIfPassed(studentInfo):
    minEnglish = 120
    minUkrainian = 120
    minMath = 120 if studentInfo['Has benefits'] else 140
    minRating = 144 if studentInfo['Has benefits'] else 160

    if (studentInfo['English'] >= minEnglish and studentInfo['Ukrainian'] >= minUkrainian and
       studentInfo['Math'] >= minMath and studentInfo['rating'] >= minRating):
        return True
    else:
        return False

In [3]:
def createDataset(numOfApplicants):
    marks = np.random.randint(100, 201, (numOfApplicants, 3))
    rating = np.array([calcRating(marks[i, 0], marks[i, 1], marks[i, 2]) for i in range(numOfApplicants)])
    count_accepted = 0
    count_benef_accepted = 0

    dataset = pd.DataFrame(marks, columns=['Math', 'English', 'Ukrainian'])
    dataset['accepted'] = 0
    dataset['rating'] = rating
    dataset['Has benefits'] = np.random.choice([0, 1], size=numOfApplicants, p=[0.9, 0.1])

    max_accepted = int(350 / 1500 * numOfApplicants)
    max_benef_accepted = int(max_accepted * 0.1)
    dataset_copy = dataset.sort_values(['Has benefits', 'rating'], ascending=[False, False])

    for i, studentInfo in dataset_copy.iterrows():
        if not studentInfo['Has benefits'] or count_benef_accepted == max_benef_accepted or count_accepted == max_accepted:
            break
        if checkIfPassed(studentInfo):
            count_accepted += 1
            count_benef_accepted += 1
            dataset.at[i, 'accepted'] = 1

    dataset_copy = dataset.sort_values(['accepted', 'rating'], ascending=[True, False])

    for i, studentInfo in dataset_copy.iterrows():
        if count_accepted == max_accepted:
            break
        if studentInfo['Has benefits']:
            continue
        if checkIfPassed(studentInfo):
            count_accepted += 1
            dataset.at[i, 'accepted'] = 1

    return dataset

In [4]:
train_dataset = createDataset(50000)
test_dataset = createDataset(1500)

In [5]:
train_dataset.head()

,Math,English,Ukrainian,accepted,rating,Has benefits
0,145,188,162,1,164.7,0
1,112,135,177,0,144.9,0
2,162,106,120,0,128.4,0
3,164,147,162,0,158.1,0
4,109,130,137,0,126.5,0


In [6]:
train_dataset.shape

(50000, 6)

In [7]:
X_train, y_train = train_dataset.drop(columns='accepted'), train_dataset['accepted']
X_test, y_test = test_dataset.drop(columns='accepted'), test_dataset['accepted']

In [8]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
X_train = mms.fit_transform(X_train)
X_test = mms.fit_transform(X_test)

X_train

array([[0.45      , 0.88      , 0.62      , 0.64559585, 0.        ],
       [0.12      , 0.35      , 0.77      , 0.44041451, 0.        ],
       [0.62      , 0.06      , 0.2       , 0.26943005, 0.        ],
       ...,
       [0.06      , 0.17      , 0.97      , 0.44870466, 0.        ],
       [0.61      , 0.17      , 0.91      , 0.59481865, 0.        ],
       [0.08      , 0.59      , 0.99      , 0.59378238, 0.        ]])

In [9]:
from keras import models
from keras import layers

In [16]:
network = models.Sequential([
    layers.Dense(4, input_dim=5, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [17]:
network.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
network.fit(X_train, y_train, epochs=6, batch_size=50)

Epoch 1/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 737us/step - accuracy: 0.7310 - loss: 0.6258
Epoch 2/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 708us/step - accuracy: 0.7747 - loss: 0.4461
Epoch 3/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 746us/step - accuracy: 0.7708 - loss: 0.3484
Epoch 4/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 719us/step - accuracy: 0.8905 - loss: 0.2779
Epoch 5/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 696us/step - accuracy: 0.9170 - loss: 0.2269
Epoch 6/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 703us/step - accuracy: 0.9285 - loss: 0.1939


In [18]:
test_loss, test_acc = network.evaluate(X_test, y_test)

47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 695us/step - accuracy: 0.9469 - loss: 0.1801


In [19]:
network = models.Sequential([
    layers.Dense(8, input_dim=5, activation='relu'),
    layers.Dense(4, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

network.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
network.fit(X_train, y_train, epochs=6, batch_size=50)

Epoch 1/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 733us/step - accuracy: 0.7720 - loss: 0.5207
Epoch 2/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 748us/step - accuracy: 0.9062 - loss: 0.2376
Epoch 3/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 744us/step - accuracy: 0.9528 - loss: 0.1609
Epoch 4/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 718us/step - accuracy: 0.9632 - loss: 0.1245
Epoch 5/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 826us/step - accuracy: 0.9679 - loss: 0.1009
Epoch 6/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 751us/step - accuracy: 0.9709 - loss: 0.0881


In [20]:
test_loss, test_acc = network.evaluate(X_test, y_test)

47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 739us/step - accuracy: 0.9717 - loss: 0.0883


In [21]:
network = models.Sequential([
    layers.Dense(8, input_dim=5, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

network.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
network.fit(X_train, y_train, epochs=6, batch_size=50)

Epoch 1/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 683us/step - accuracy: 0.7775 - loss: 0.5554
Epoch 2/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 704us/step - accuracy: 0.9139 - loss: 0.2447
Epoch 3/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 717us/step - accuracy: 0.9339 - loss: 0.1702
Epoch 4/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 689us/step - accuracy: 0.9403 - loss: 0.1482
Epoch 5/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 679us/step - accuracy: 0.9439 - loss: 0.1362
Epoch 6/6
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 692us/step - accuracy: 0.9495 - loss: 0.1232


In [22]:
test_loss, test_acc = network.evaluate(X_test, y_test)

47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 674us/step - accuracy: 0.9591 - loss: 0.1145


In [23]:
network.save('my_model.keras')

In [24]:
predictions = network.predict(X_test)
predictions

47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


array([[1.2086210e-07],
       [5.3106462e-05],
       [2.5963467e-01],
       ...,
       [9.9911296e-01],
       [9.8477519e-01],
       [7.5124652e-04]], dtype=float32)

In [26]:
result = test_dataset
result['accepted'] = np.round(predictions)

In [28]:
result.to_excel('result.xlsx')